# Exemplo de Uso – Vassoura 

Este notebook mostra como utilizar as principais funcionalidades do pacote `vassoura` para detectar e limpar multicolinearidade, além de gerar relatórios visuais.

In [ ]:
import numpy as np
import pandas as pd
import vassoura as vs

## Criando um DataFrame de exemplo com colinearidade artificial

In [ ]:
rng = np.random.default_rng(42)
n = 300
df = pd.DataFrame({
    'x1': rng.normal(0, 1, n),
})
df['x2'] = df['x1'] * 0.95 + rng.normal(0, 0.05, n)
df['x3'] = rng.normal(0, 1, n)
df['cat'] = rng.choice(['A', 'B', 'C'], size=n)
df['target'] = ((df['x1'] + df['x3']) > 0).astype(int)
df.head()

## Detectando e limpando multicolinearidade

In [ ]:
df_limpo, removidas, matriz_corr, vif_df = vs.clean_multicollinearity(
    df,
    target_col='target',
    include_target=False,
    keep_cols=['x1'],
    corr_threshold=0.9,
    vif_threshold=5,
    verbose=True
)

In [ ]:
print('Variáveis removidas:', removidas)

## Geração de relatório com gráficos e análise com/sem target

In [ ]:
vs.generate_report(df, target_col='target', output_path='vassoura_report.html')

Abra o arquivo `vassoura_report.html` para visualizar o relatório gerado.